# Summary

Best ROC-AUC (base) was 86.47% on Random Forest. With TPOT, ROC-AUC is 89.53%. With Neural Network TPOT, ROC-AUC is 94.50%. 


* TPOT stands for Tree-based Pipeline Optimization Tool. TPOT uses genetic programming to find the optimal ML pipeline 
* TPOT requires data to be numerical. Since we have preprocessed data already, we will move on to TPOT deployment directly
* TPOT takes 1D arrays, therefore dataframes are raveled
* Best models are be exported as py files


In [ ]:
! pip install tpot
! pip install ipywidgets jupyter nbextension enable --py widgetsnbextension
! pip install dask dask-ml

In [7]:
from tpot import TPOTClassifier
import pandas as pd 

In [8]:
# import preprocessed data
X_train = pd.read_csv('/Users/chiaralu/Desktop/Courses/INSY 695/Group Project/hotel_cancellation_ML2/datasets/X_train.csv')
y_train = pd.read_csv('/Users/chiaralu/Desktop/Courses/INSY 695/Group Project/hotel_cancellation_ML2/datasets/y_train.csv')
X_test = pd.read_csv('/Users/chiaralu/Desktop/Courses/INSY 695/Group Project/hotel_cancellation_ML2/datasets/X_test.csv')
y_test = pd.read_csv('/Users/chiaralu/Desktop/Courses/INSY 695/Group Project/hotel_cancellation_ML2/datasets/y_test.csv')
X_val = pd.read_csv('/Users/chiaralu/Desktop/Courses/INSY 695/Group Project/hotel_cancellation_ML2/datasets/X_val.csv')
y_val = pd.read_csv('/Users/chiaralu/Desktop/Courses/INSY 695/Group Project/hotel_cancellation_ML2/datasets/y_val.csv')


In [9]:
# reshape to 1d array

X_train.values.ravel()
y_train.values.ravel()
X_test.values.ravel()
y_test.values.ravel()
X_val.values.ravel()
y_val.values.ravel()

array([0, 1, 1, ..., 0, 0, 0])

In [23]:
tpot = TPOTClassifier(generations=20, population_size=20, mutation_rate = 0.05, verbosity=2, scoring = 'roc_auc',
                      cv = 5, n_jobs=-1, max_time_mins = 1, max_eval_time_mins = 3,
                      random_state=42, config_dict='TPOT light')
tpot.fit(X_train, y_train)

/Applications/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Optimization Progress:   0%|          | 0/20 [00:00<?, ?pipeline/s]


3.31 minutes have elapsed. TPOT will close down.
TPOT closed during evaluation in one generation.


TPOT closed prematurely. Will use the current best pipeline.

Best pipeline: DecisionTreeClassifier(input_matrix, criterion=gini, max_depth=7, min_samples_leaf=20, min_samples_split=4)


TPOTClassifier(config_dict='TPOT light', generations=20, max_eval_time_mins=3,
               max_time_mins=1, mutation_rate=0.05, n_jobs=-1,
               population_size=20, random_state=42, scoring='roc_auc',
               verbosity=2)

In [25]:
print(f"ROC-AUC score: {tpot.score(X_test, y_test)}")

ROC-AUC score: 0.8953535723254082


/Applications/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [27]:
tpot.export('tpot_pipeline.py')

## Neural network classifier using TPOT-NN

In [ ]:
from tpot import TPOTClassifier
from sklearn.datasets import make_blobs

In [20]:
nn_tpot = TPOTClassifier(config_dict='TPOT NN', 
                     verbosity=2, population_size=5, generations=5, n_jobs = -1,max_time_mins = 2, 
                     scoring = 'roc_auc')

assert not hasattr(nn_tpot, "classes_")
nn_tpot.fit(X_train, y_train)
assert hasattr(nn_tpot, "classes_")

/Applications/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Optimization Progress:   0%|          | 0/5 [00:00<?, ?pipeline/s]


4.15 minutes have elapsed. TPOT will close down.
TPOT closed during evaluation in one generation.


TPOT closed prematurely. Will use the current best pipeline.

Best pipeline: RandomForestClassifier(input_matrix, bootstrap=True, criterion=gini, max_features=0.5, min_samples_leaf=1, min_samples_split=19, n_estimators=100)


In [21]:
print(f"AUC-ROC score: {nn_tpot.score(X_test, y_test)}")

/Applications/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


AUC-ROC score: 0.944954449228998


In [22]:
# export best nn_tpot
nn_tpot.export('nn_tpot_pipeline.py')